In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pickle
from time import time

import LimitedCommitmentModel as lcm

path = 'output/'

# c++ settings
do_compile = True
threads = 20

# from EconModel import cpptools
# cpptools.setup_nlopt(folder='cppfuncs/', do_print=True)

# Solve and simulate from alternative models

Benchmark model settings

In [2]:
settings = { 
       'T':20,
       'p_meet': 0.1, 
       'div_A_share': 0.5, 
       'sigma_love':0.1,
       'threads':threads, 
       'interp_power':False,
       'num_love': 21, 
       'do_egm':True, 
       'num_A': 50, 
       'num_A_pd':50,
       'num_Ctot':50,
       'simT':20,
       'simN': 10000,
       'centered_gradient':True,
       'use_external_solution':True,
       }

Solve "true" model through dense grids using VFI

In [3]:
specs_true = {'true': {'latexname':'true', 'par':{**settings, 'do_egm':False, 'num_A': 250, 'num_Ctot':250, 'num_power':101, 'num_love':101}}}

model_true = lcm.HouseholdModelClass(par=specs_true['true']['par'])
model_true.link_to_cpp(force_compile=do_compile)
model_true.solve()

In [4]:
# model = lcm.HouseholdModelClass(par=settings)
# model.link_to_cpp(force_compile=False)

# model.par.do_egm = True
# model.par.interp_method = 'numerical'

# model.par.precompute_intratemporal = True
# # model.par.interp_intra_period_in_num_inverse = model.par.precompute_intratemporal
# %time model.solve()

# model.par.precompute_intratemporal = False
# # model.par.interp_intra_period_in_num_inverse = model.par.precompute_intratemporal
# %time model.solve()

In [5]:
# stop

## Monte Carlo runs

Monte Carlo settings

In [6]:
MC_num = 1 # number of Monte Carlo simulations
C_num_grid = (20,50,100,200) # number of grid points in consumption grid i iEGM
do_EGM = True
do_VFI = True
PRINT = True

Set up containers

In [7]:
timing = {
    'vfi':np.nan + np.zeros(MC_num),
    'vfi no precomp':np.nan + np.zeros(MC_num),
    'egm': np.nan + np.zeros(MC_num),
    'egm no precomp':np.nan + np.zeros(MC_num),
    'iegm, linear':dict(),
    'iegm, linear inverse':dict(),
}
util = {
    'vfi':np.nan + np.zeros(MC_num),
    'vfi no precomp':np.nan + np.zeros(MC_num),
    'egm': np.nan + np.zeros(MC_num),
    'egm no precomp':np.nan + np.zeros(MC_num),
    'iegm, linear':dict(),
    'iegm, linear inverse':dict(),
}
for i_c,num_C in enumerate(C_num_grid):
    timing['iegm, linear'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear'][num_C] = np.nan + np.zeros(MC_num)

    timing['iegm, linear inverse'][num_C] = np.nan + np.zeros(MC_num)
    util['iegm, linear inverse'][num_C] = np.nan + np.zeros(MC_num)

Run monte carlo simulations

In [8]:
for i_mc in range(MC_num):
    if PRINT: print(f'{i_mc+1}/{MC_num} running...')

    # simulate true model (solved once above)
    model_true.par.seed = i_mc
    model_true.allocate_draws()
    model_true.simulate()
    true_mean_lifetime_util = model_true.sim.mean_lifetime_util

    # setup alternative model solutios
    model = lcm.HouseholdModelClass(par=settings)

    model.par.seed = i_mc
    model.allocate()
    model.link_to_cpp(force_compile=False)

    # VFI
    if do_VFI:
        model.par.do_egm = False
        for precomp in (True,False):
            model.par.precompute_intratemporal = precomp
            name_now = 'vfi' if precomp else 'vfi no precomp'

            # Timing
            t0 = time()
            model.solve()
            timing[name_now][i_mc] = time() - t0

            # Lifetime utility error
            model.simulate()
            util[name_now][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100


    # EGM, numerical
    if do_EGM:
        model.par.do_egm = True
        model.par.interp_method = "numerical"
        for precomp in (True,False):
            model.par.precompute_intratemporal = precomp
            name_now = 'egm' if precomp else 'egm no precomp'

            # Timing
            t0 = time()
            model.solve()
            timing[name_now][i_mc] = time() - t0

            # Lifetime utility error
            model.simulate()
            util[name_now][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100


    # iEGM
    model.par.do_egm = True
    model.par.interp_method = "linear"
    model.par.precompute_intratemporal = True # does not matter here now (if switch is also used to determine wheter this object is calculated, set it to of here for time saving)
    for interp_inverse in (False,True):
        model.par.interp_inverse = interp_inverse
        method = f'iegm, linear inverse' if interp_inverse else 'iegm, linear'
        for i_c,num_C in enumerate(C_num_grid):
            model.par.num_marg_u = num_C

            model.allocate()

            # Timing
            t0 = time()
            model.solve()
            timing[method][num_C][i_mc] = time() - t0

            # Euler error
            model.simulate()
            util[method][num_C][i_mc] = np.abs((model.sim.mean_lifetime_util - true_mean_lifetime_util)/true_mean_lifetime_util) * 100
            
    model.cpp.delink()

    # save MC objects
    # with open('output/MC_timing.pkl', 'wb') as f:
    #     pickle.dump(timing, f)
    # with open('output/MC_util.pkl', 'wb') as f:
    #     pickle.dump(util, f)

1/1 running...


Results

In [13]:
print('Lifetime utility & Timing (rel. to VFI)')
timing_vfi = np.mean(timing['vfi'])
# for method in ('vfi',):
for method in ('vfi','vfi no precomp','egm','egm no precomp'):
    util_now = np.nanmean(util[method])
    time_now = np.nanmean(timing[method]) / timing_vfi
    print(f'{method}: {util_now:2.2f} & {time_now:2.2f} ')

for method in ('iegm, linear','iegm, linear inverse'):
    print(f'{method}: ')
    for i_c,num_C in enumerate(C_num_grid):
        util_now = np.nanmean(util[method][num_C]) 
        time_now = np.nanmean(timing[method][num_C]) / timing_vfi
        print(f'{num_C:d} {util_now:2.2f} & {time_now:2.2f} ')

# Lifetime utility & Timing (rel. to VFI)
# vfi: 0.527 & 1.000 
# iegm, linear: 
# 20 0.466 & 0.143 
# 50 0.518 & 0.130 
# 100 0.527 & 0.148 
# 200 0.527 & 0.149 
# iegm, linear inverse: 
# 20 0.501 & 0.130 
# 50 0.527 & 0.117 
# 100 0.527 & 0.158 
# 200 0.527 & 0.175 

Lifetime utility & Timing (rel. to VFI)
vfi: 0.52 & 1.00 
vfi no precomp: 0.52 & 296.30 
egm: 0.52 & 0.87 
egm no precomp: 0.52 & 78.71 
iegm, linear: 
20 0.45 & 0.07 
50 0.50 & 0.07 
100 0.52 & 0.08 
200 0.52 & 0.10 
iegm, linear inverse: 
20 0.49 & 0.07 
50 0.52 & 0.07 
100 0.52 & 0.08 
200 0.52 & 0.10 
